# 广义函数(generic function)

In [1]:
; CLOS: Common Lisp Object System
; T: 根类, 所有其他类的直接或间接基类
; Common Lisp支持多重继承
; 广义函数: 统一了方法(method)和函数

## DEFGENERIC

In [ ]:
; 广义函数定义了抽象操作, 指定了名字和一个参数列表, 但不提供实现
; 广义函数的实际实现是由方法提供的, 每个方法提供了广义函数用于特定参数类的实现.

In [ ]:
(defgeneric draw (shape)
    (:documentation "Draw the given shape on the screen."))

# 类

## DEFCLASS

In [2]:
; DEFCLASS的职责仅仅是将类定义为一种数据类型
; 类的名字与函数和变量的名字处在独立的命名空间中
;
; (defclass name (direct-superclass-name*)
;   (slot-specifier*))
; 如果没有irect-superclass-name, 则直接成为STANDARD-OBJECT的子类(STANDARD-OBJECT是T的子类) 
;
; MAKE-INSTANCE: 创建用户定义类的新实例

## 槽描述符

In [3]:
; 实例中的每个槽是一个可以保存值的位置
; 该位置可以通过函数SLOT-VALUE来访问, SLOT-VALUE组合SETF来设置槽的值
; 一个类可以从它的所有基类中继承槽描述符

In [4]:
(defclass bank-account ()
    (customer-name
     balance))

#<STANDARD-CLASS COMMON-LISP-USER::BANK-ACCOUNT>

In [7]:
; 对象的打印形式取决于广义函数PRINT-OBJECT
(make-instance 'bank-account)

#<BANK-ACCOUNT {1004424D03}>

In [8]:
; 设置和访问槽
(defparameter *account* (make-instance 'bank-account))
(setf (slot-value *account* 'customer-name) "John Doe")
(setf (slot-value *account* 'balance) 1000)
(slot-value *account* 'customer-name)
(slot-value *account* 'balance)

*ACCOUNT*

"John Doe"

1000

"John Doe"

1000

## 对象初始化

In [ ]:
; 三种方式控制槽的初始值
; (1) :initarg: 指定MAKE-INSTANCE的关键字形参
; (2) :initform: 指定在没有:initarg参数时用一个Lisp表达式为该槽计算一个值
; (3) 在广义函数INITIALIZE-INSTANCE上定义一个方法, 它将被MAKE-INSTANCE调用

In [10]:
(defclass bank-account ()
    ((customer-name
      :initarg :customer-name)
     (balance
      :initarg :balance
      :initform 0)))

#<STANDARD-CLASS COMMON-LISP-USER::BANK-ACCOUNT>

In [11]:
(defparameter *account*
    (make-instance 'bank-account :customer-name "John Doe" :balance 1000))
(slot-value *account* 'customer-name)
(slot-value *account* 'balance)

*ACCOUNT*

"John Doe"

1000

In [13]:
(slot-value (make-instance 'bank-account) 'balance)
; The slot COMMON-LISP-USER::CUSTOMER-NAME is unbound in the object
; (slot-value (make-instance 'bank-account) 'customer-name)

0

In [14]:
(defvar *account-numbers* 0)

(defclass bank-account ()
    ((customer-name
      :initarg :customer-name
      :initform (error "Must supply a customer name"))
     (balance
      :initarg :balance
      :initform 0)
     (account-number
      :initform (incf *account-numbers*))))

*ACCOUNT-NUMBERS*

#<STANDARD-CLASS COMMON-LISP-USER::BANK-ACCOUNT>

In [15]:
; REQ: 基于一个槽的值来初始化另一个槽

In [16]:
(defclass bank-account ()
    ((customer-name
      :initarg :customer-name
      :initform (error "Must supply a customer name"))
     (balance
      :initarg :balance
      :initform 0)
     (account-number
      :initform (incf *account-numbers*))
     account-type))

;; 根据balance设置account-type
;; STANDARD-OBJECT上特化的INITIALIZE-INSTANCE主方法负责槽的初始化工作
(defmethod initialize-instance :after ((account bank-account) &key)
    (let ((balance (slot-value account 'balance)))
         (setf (slot-value account 'account-type)
               (cond
                   ((>= balance 100000) :gold)
                   ((>= balance 50000) :silver)
                   (t :bronze)))))

(slot-value (make-instance 'bank-account 
                           :customer-name "John Doe"
                           :balance 200000) 'account-type)

#<STANDARD-CLASS COMMON-LISP-USER::BANK-ACCOUNT>

#<STANDARD-METHOD COMMON-LISP:INITIALIZE-INSTANCE :AFTER (BANK-ACCOUNT) {10036D1993}>

:GOLD

In [25]:
(defclass bank-account ()
    ((customer-name
      :initarg :customer-name
      :initform (error "Must supply a customer name"))
     (balance
      :initarg :balance
      :initform 0)
     (account-number
      :initform (incf *account-numbers*))))

;; 移除主方法
(remove-method #'initialize-instance
               (find-method #'initialize-instance '(:after) (list (find-class 'bank-account))))

;; 指定了一个&key参数
(defmethod initialize-instance :after ((account bank-account) 
                                       &key opening-bonus-percentage)
    (when opening-bonus-percentage
        (incf (slot-value account 'balance)
              (* (slot-value account 'balance) (/ opening-bonus-percentage 100)))))

(slot-value (make-instance 'bank-account
                           :customer-name "Sally Sue"
                           :balance 1000
                           :opening-bonus-percentage 5) 'balance)

#<STANDARD-CLASS COMMON-LISP-USER::BANK-ACCOUNT>

#<STANDARD-GENERIC-FUNCTION COMMON-LISP:INITIALIZE-INSTANCE (40)>

#<STANDARD-METHOD COMMON-LISP:INITIALIZE-INSTANCE :AFTER (BANK-ACCOUNT) {1004F17CD3}>

1050

## 访问函数

In [26]:
; TODO(zhoujiagen) SETF函数
; 
; 槽选项: :read, :write, :accessor, :documentation

In [38]:
(defclass bank-account ()
    ((customer-name
      :initarg :customer-name
      :initform (error "Must supply a customer name")
      :accessor customer-name
      :documentation "Customer's name")
      (balance
       :initarg :balance
       :initform 0
       :reader balance
       :documentation "Current account balance")
      (account-number
       :initform (incf *account-numbers*)
       :reader account-number
       :documentation "Account number, unique within a bank.")
      (account-type
       :reader account-type
       :documentation "Type of acount, one of :gold, :silver, or :bronze.")))

(defmethod initialize-instance :after ((account bank-account) &key)
    (let ((balance (slot-value account 'balance)))
         (setf (slot-value account 'account-type)
               (cond
                   ((>= balance 100000) :gold)
                   ((>= balance 50000) :silver)
                   (t :bronze)))))

(defparameter *account* (make-instance 'bank-account 
                                       :customer-name "John Doe"))
(customer-name *account*)
(balance *account*)
(account-number *account*)
(account-type *account*)

(setf (customer-name *account*) "John Smith")
(customer-name *account*)

#<STANDARD-CLASS COMMON-LISP-USER::BANK-ACCOUNT>

#<STANDARD-METHOD COMMON-LISP:INITIALIZE-INSTANCE :AFTER (BANK-ACCOUNT) {10042FBE83}>

*ACCOUNT*

"John Doe"

0

19

:BRONZE

"John Smith"

"John Smith"


REDEFINITION-WITH-DEFMETHOD: 
  redefining INITIALIZE-INSTANCE :AFTER (#<STANDARD-CLASS COMMON-LISP-USER::BANK-ACCOUNT>) in DEFMETHOD


## WITH-SLOTS, WITH-ACCESSORS
